<a href="https://colab.research.google.com/github/luciasalmeron/TFM_Ingenieria_Biomedica_y_Salud_Digital/blob/main/Evaluacion_de_modelos_basico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Montaje de Google Drive y Preparación del Entorno

In [ ]:
from google.colab import drive
# Monta Google Drive
drive.mount('/content/drive')

# https://api.isic-archive.com/collections/65/

Mounted at /content/drive


In [ ]:
!pip install tensorflow
!pip install split-folders

Imports

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping

import os
import pandas as pd
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import splitfolders
import tensorflow as tf
from datetime import datetime
import time

#2. Selección del Modelo de Arquitectura

In [ ]:
## SELECT THE DESIRED MODEL ARCHITECTURE ##
#from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as model, preprocess_input
#from tensorflow.keras.applications.resnet50 import ResNet50 as model, preprocess_input
#from tensorflow.keras.applications.vgg16 import VGG16 as model, preprocess_input
#from tensorflow.keras.applications.inception_v3 import InceptionV3 as model, preprocess_input
##from tensorflow.keras.applications.densenet121 import DenseNet121 as model, preprocess_input ME DA ERROR AL EJECUTARLO
#from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L as model, preprocess_input
#from tensorflow.keras.applications.xception import Xception as model, preprocess_input
from tensorflow.keras.applications.convnext import ConvNeXtLarge as model, preprocess_input

from tensorflow.keras.metrics import Accuracy, AUC, Precision, Recall, TruePositives, TrueNegatives, FalsePositives, FalseNegatives

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/ISIC-images2020trainingg'

#3. Configuración de Parámetros y Directorios

In [ ]:
NUM_CLASSES = 2
CLASSES = ['Benign', 'Malignant']
##MALIGNANT -->  POSITIVA
##BENIGN --> NEGATIVA

## Training variables ##
INPUT_SIZE = 224
BATCH_SIZE = 32  # size of the readed batches from generator, must fit on memory
VAL_SPLIT = 0.15  # fraction of the images used for validation
TEST_SPLIT = 0.15  # fraction of the images used for testing
EPOCHS = 20

image_folder = '/content/drive/My Drive/ISIC-images2020trainingg'
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]


# Crear las carpetas "benigno" y "maligno" si no existen
BASE_DATASET = '/content/drive/My Drive/imagenes520/imagenes_clasificadas'   # Carpeta base con las carpetas "benigno" y "maligno"
benigno_folder = BASE_DATASET+'/clasificadas_benigno/'
maligno_folder = BASE_DATASET+'/clasificadas_maligno/'

SPLITTED_DATASET = 'splitted_dataset'  # Carpeta para los datos divididos
SAVE_MODELS_PATH = '/content/drive/My Drive/trained_models'  # Carpeta para guardar los modelos



#4. Visualización de Imágenes Iniciales

In [ ]:
# Lista los archivos en la carpeta
print(f"Se encontraron {len(image_files)} imágenes.")

 Mostrar las primeras 5 imágenes
for img_file in image_files[:5]:
    img_path = os.path.join(image_folder, img_file)
    img = Image.open(img_path)
    plt.imshow(img)
    plt.title(img_file)
    plt.axis('off')
    plt.show()'''

#5. Clasificación de Imágenes según Etiquetas

In [ ]:
# Leer el archivo Excel
df = pd.read_csv("/content/drive/My Drive/challenge-training_metadata.csv", sep=";")

# Crear las carpetas si no existen
os.makedirs(benigno_folder, exist_ok=True)
os.makedirs(maligno_folder, exist_ok=True)

# Contadores
benigno_count = 0
maligno_count = 0
no_encontrado_count = 0

# Clasificar las imágenes
for index, row in df.iterrows():
    image_name = row['isic_id'] + '.jpg'
    label = row['diagnosis_1']
    image_path = os.path.join(image_folder, image_name)

    if os.path.exists(image_path):
        if label == 'Benign':
            shutil.move(image_path, os.path.join(benigno_folder, image_name))
            benigno_count += 1
        elif label == 'Malignant':
            shutil.move(image_path, os.path.join(maligno_folder, image_name))
            maligno_count += 1
    else:
        #print(f"Imagen no encontrada: {image_name}")
        no_encontrado_count += 1

    # Mostrar progreso en tiempo real
    print(f"Movidas - Benigno: {benigno_count}, Maligno: {maligno_count}, No encontradas: {no_encontrado_count}")


'\n\n# Crear las carpetas si no existen\nos.makedirs(benigno_folder, exist_ok=True)\nos.makedirs(maligno_folder, exist_ok=True)\n\n# Contadores\nbenigno_count = 0\nmaligno_count = 0\nno_encontrado_count = 0\n\n# Clasificar las imágenes\nfor index, row in df.iterrows():\n    image_name = row[\'isic_id\'] + \'.jpg\'\n    label = row[\'diagnosis_1\']\n    image_path = os.path.join(image_folder, image_name)\n\n    if os.path.exists(image_path):\n        if label == \'Benign\':\n            shutil.move(image_path, os.path.join(benigno_folder, image_name))\n            benigno_count += 1\n        elif label == \'Malignant\':\n            shutil.move(image_path, os.path.join(maligno_folder, image_name))\n            maligno_count += 1\n    else:\n        #print(f"Imagen no encontrada: {image_name}")\n        no_encontrado_count += 1\n\n    # Mostrar progreso en tiempo real\n    print(f"Movidas - Benigno: {benigno_count}, Maligno: {maligno_count}, No encontradas: {no_encontrado_count}")\n\n# R

#6. Comprobación de Conteo de Imágenes

In [ ]:
# Resumen final
print("\nProceso finalizado.")
print(f"Total movidas a Benigno: {benigno_count}")
print(f"Total movidas a Maligno: {maligno_count}")
print(f"Total imágenes no encontradas: {no_encontrado_count}")

In [ ]:

# Contar las imágenes en cada carpeta
benigno_count = len([f for f in os.listdir(benigno_folder) if os.path.isfile(os.path.join(benigno_folder, f))])
maligno_count = len([f for f in os.listdir(maligno_folder) if os.path.isfile(os.path.join(maligno_folder, f))])

# Mostrar los resultados
print(f"Número de imágenes en la carpeta 'benigno': {benigno_count}")
print(f"Número de imágenes en la carpeta 'maligno': {maligno_count}")

# Contar las imágenes en la carpeta original
original_count = len([f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))])

# Mostrar el resultado
print(f"Número de imágenes en la carpeta original 'image_folder': {original_count}")


'\n# Contar las imágenes en cada carpeta\nbenigno_count = len([f for f in os.listdir(benigno_folder) if os.path.isfile(os.path.join(benigno_folder, f))])\nmaligno_count = len([f for f in os.listdir(maligno_folder) if os.path.isfile(os.path.join(maligno_folder, f))])\n\n# Mostrar los resultados\nprint(f"Número de imágenes en la carpeta \'benigno\': {benigno_count}")\nprint(f"Número de imágenes en la carpeta \'maligno\': {maligno_count}")\n\n# Contar las imágenes en la carpeta original\noriginal_count = len([f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))])\n\n# Mostrar el resultado\nprint(f"Número de imágenes en la carpeta original \'image_folder\': {original_count}")\n'

#7. División del Dataset

In [ ]:
# Dividir el dataset en entrenamiento, validación y prueba
if not os.path.exists(SPLITTED_DATASET):
    splitfolders.ratio(BASE_DATASET, output=SPLITTED_DATASET, seed=123, ratio=(1 - VAL_SPLIT - TEST_SPLIT, VAL_SPLIT, TEST_SPLIT))


Se han truncado las últimas 5000 líneas del flujo de salida.
Copying files: 43838 files [8:33:59,  1.47 files/s]
Copying files: 43839 files [8:33:59,  1.40 files/s]
Copying files: 43840 files [8:34:00,  1.44 files/s]
Copying files: 43841 files [8:34:01,  1.58 files/s]
Copying files: 43842 files [8:34:01,  1.53 files/s]
Copying files: 43843 files [8:34:02,  1.63 files/s]
Copying files: 43844 files [8:34:02,  1.60 files/s]
Copying files: 43845 files [8:34:03,  1.59 files/s]
Copying files: 43846 files [8:34:04,  1.58 files/s]
Copying files: 43847 files [8:34:04,  1.53 files/s]
Copying files: 43848 files [8:34:05,  1.49 files/s]
Copying files: 43849 files [8:34:06,  1.49 files/s]
Copying files: 43850 files [8:34:07,  1.47 files/s]
Copying files: 43851 files [8:34:07,  1.52 files/s]
Copying files: 43852 files [8:34:08,  1.50 files/s]
Copying files: 43853 files [8:34:09,  1.42 files/s]
Copying files: 43854 files [8:34:09,  1.44 files/s]
Copying files: 43855 files [8:34:10,  1.46 files/s]
Cop

#8. Conteo por Conjunto

In [ ]:
def count_images_in_folders(base_path):
    folder_counts = {}
    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path):
            num_images = len([f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif'))])
            folder_counts[folder] = num_images
    return folder_counts

training_path = 'splitted_dataset/train'
validation_path = 'splitted_dataset/val'

training_counts = count_images_in_folders(training_path)
validation_counts = count_images_in_folders(validation_path)


Pesos de clase: {0: 0.593300167790174, 1: 3.1795235841615397}
Training set image counts:
clasificadas_maligno: 6339 images
clasificadas_benigno: 33971 images

Validation set image counts:
clasificadas_maligno: 1358 images
clasificadas_benigno: 7279 images


#9. Cálculo de Pesos de Clase

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
# Definir clases
classes = np.array([0, 1])  # 0 = Benigno, 1 = Maligno


num_benign = training_counts["clasificadas_benigno"]
num_malignant = training_counts["clasificadas_maligno"]

# Calcular los pesos de cada clase
class_weights = compute_class_weight(class_weight="balanced", classes=classes,
                                     y=np.concatenate([np.zeros(num_benign), np.ones(num_malignant)]))

# Convertir a diccionario
class_weights = {i: weight for i, weight in enumerate(class_weights)}

print("Pesos de clase:", class_weights)


print("Training set image counts:")
for folder, count in training_counts.items():
    print(f"{folder}: {count} images")

print("\nValidation set image counts:")
for folder, count in validation_counts.items():
    print(f"{folder}: {count} images")

#10. Generadores de Imágenes

In [ ]:
# Generadores de imágenes
#datagen = ImageDataGenerator(rescale=1.0/255)

def preprocess(images):
      # Using the preprocess function of the selected model
      # To ensure the new data is in the same format as the original data the model was trained on
      return preprocess_input(images)




seed=123

train_datagen = ImageDataGenerator(#rescale=1./255,
                                    #rotation_range = 25,
                                    #shear_range=0.2,
                                    #zoom_range=0.2,
                                    #width_shift_range=0.0,
                                    #height_shift_range=0.0,
                                    #brightness_range=None,
                                    #horizontal_flip=True,
                                    #vertical_flip=True,
                                    fill_mode='wrap',
                                    preprocessing_function=preprocess)

train_generator = train_datagen.flow_from_directory(
    training_path,
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

validation_generator = train_datagen.flow_from_directory(
    validation_path,
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)


Found 40310 images belonging to 2 classes.
Found 8637 images belonging to 2 classes.


#11. Visualización de Imágenes del Generador

In [ ]:
images, labels = train_generator.__next__()
print('Images is shape', images.shape)

for i in range(4):
    f = plt.figure(figsize = (20, 20))
    f.add_subplot(1,3,1)
    plt.axis('off')
    plt. title('Label: '+str(CLASSES[int(labels[i])]))
    #plt.imshow(images[i])
    plt.imshow(images[i].astype('uint8'),vmin=0, vmax=1)

plt.show(block=True)
plt.show

"images, labels = train_generator.__next__()\nprint('Images is shape', images.shape)\n\nfor i in range(4):\n    f = plt.figure(figsize = (20, 20))\n    f.add_subplot(1,3,1)\n    plt.axis('off')\n    plt. title('Label: '+str(CLASSES[int(labels[i])]))\n    #plt.imshow(images[i])\n    plt.imshow(images[i].astype('uint8'),vmin=0, vmax=1)\n\nplt.show(block=True)\nplt.show"

#12. Construcción del Modelo CNN


In [ ]:
##########          CNN Construction           ##########

INPUT_SHAPE = (INPUT_SIZE, INPUT_SIZE, 3)

base_model = model(input_shape=INPUT_SHAPE,
                   include_top=False,
                   weights='imagenet')


# Guarda el nombre del modelo en una variable
model_name = model.__name__


base_model.trainable = False

model = tf.keras.Sequential([
                          base_model,
                          tf.keras.layers.GlobalAveragePooling2D(),
                          tf.keras.layers.Dense(1, activation='sigmoid')
                        ])


metrics = ['accuracy', 'AUC', 'Precision', 'Recall', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives']


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=metrics)



785596384/785596384 ━━━━━━━━━━━━━━━━━━━━ 26s 0us/step


#13. Entrenamiento del Modelo

In [ ]:
# Callbacks

early_stop = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Entrenar el modelo

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    class_weight=class_weights,  # Aplicamos pesos de clases
    validation_data=validation_generator,
    callbacks=[early_stop]
)


# Evaluar el modelo
#test_loss, test_accuracy = model.evaluate(test_generator)
#print(f"Accuracy en el conjunto de prueba: {test_accuracy}")


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1260/1260 ━━━━━━━━━━━━━━━━━━━━ 1239s 963ms/step - AUC: 0.8910 - FalseNegatives: 529.1134 - FalsePositives: 2884.2205 - Precision: 0.4595 - Recall: 0.8096 - TrueNegatives: 14098.6953 - TruePositives: 2675.5439 - accuracy: 0.8191 - loss: 0.4044 - val_AUC: 0.9398 - val_FalseNegatives: 156.0000 - val_FalsePositives: 1098.0000 - val_Precision: 0.5226 - val_Recall: 0.8851 - val_TrueNegatives: 6181.0000 - val_TruePositives: 1202.0000 - val_accuracy: 0.8548 - val_loss: 0.3275
Epoch 2/20
1260/1260 ━━━━━━━━━━━━━━━━━━━━ 1206s 957ms/step - AUC: 0.9345 - FalseNegatives: 403.5948 - FalsePositives: 2541.3188 - Precision: 0.5157 - Recall: 0.8686 - TrueNegatives: 14472.0693 - TruePositives: 2773.2791 - accuracy: 0.8518 - loss: 0.3239 - val_AUC: 0.9427 - val_FalseNegatives: 192.0000 - val_FalsePositives: 879.0000 - val_Precision: 0.5702 - val_Recall: 0.8586 - val_TrueNegatives: 6400.0000 - val_TruePositives: 1166.0000 - val_accuracy: 0.8760 - val_loss: 0.2795
Epoch 3/20
1260/1260 ━━━━━━━━━━━━━━━━━━━━ 12

#14. Guardado del Modelo y Resultados

In [ ]:
# Guardar el modelo

# Obtener la mejor precisión de validación
best_val_accuracy = max(history.history['val_accuracy'])  # Mejor val_accuracy alcanzada
best_val_accuracy_str = f"{best_val_accuracy:.4f}"  # Formatear 4 decimales

# Obtener timestamp sin espacios
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Crear el nombre del archivo con timestamp y val_accuracy
filename = f"Modelo_{model_name}_{timestamp}_valacc{best_val_accuracy_str}.h5"

# Crear carpetas para modelos y logs
os.makedirs(SAVE_MODELS_PATH, exist_ok=True)
model.save(os.path.join(SAVE_MODELS_PATH, filename))

# Convertir el history a DataFrame
history_df = pd.DataFrame(history.history)

# Guardar en un CSV
history_filename = f"History_{model_name}_{timestamp}_valacc{best_val_accuracy_str}.csv"
history_df.to_csv(os.path.join(SAVE_MODELS_PATH, history_filename), index=False)



#15. Gráficas de Métricas de Entrenamiento

In [ ]:
##########           Model evaluation           ##########

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.p
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.show()

max_iou = max(val_acc)
print ("Maximum validation accuracy reached: ", max_iou)
max_index = val_acc.index(max_iou)
print("Maximum validation accuracy reached at epoch: ",max_index+1)'''

#16. Evaluación con el Conjunto de Test

In [ ]:
##########           Model test           ##########
from keras.models import load_model

path = SAVE_MODELS_PATH+"/aaa.h5"
model = load_model(path, compile=True)


def preprocess(images):
  return preprocess_input(images)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess)

test_generator = test_datagen.flow_from_directory(
    os.path.join(SPLITTED_DATASET, 'test'),
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)


# Evaluate on Validation data
scores = model.evaluate(test_generator)


for name, value in zip(model.metrics_names, scores):
  print(name, ': ', value)